# Read spreadsheets from David and update database
These spreadsheets were imported on February 2022, I need to update the database.
## load libraries and DB connection parameters

In [2]:
import openpyxl
from pathlib import Path
import os

In [3]:
repodir = Path("../") 
inputdir = repodir / "data/"

Database credentials are stored in a database.ini file

In [4]:
from configparser import ConfigParser
import psycopg2

filename = repodir / 'secrets' / 'database.ini'
section = 'aws-lght-sl'

# create a parser
parser = ConfigParser()
# read config file
parser.read(filename)

# get section, default to postgresql
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

    

## Read spreadsheets
Each spreadsheet has a slightly different structure, so these scripts have to be adapted for each case.
### Newnes Plateau Shrub Swamp

In [5]:
inputfile=inputdir / "field-form" / "Fire response quadrat survey Newnes Nov2020_DK_revised IDs+AllNovData.xlsm"
# using data_only=True to get the calculated cell values
wb = openpyxl.load_workbook(inputfile,data_only=True)

In [6]:
ws = wb['BS1']

In [7]:
ws['A1'].value

'Site information'

In [8]:
ws.cell(row=29, column=1).value

'Quadrat #'

In [61]:
ws = wb['Site']

In [62]:
ws

<Worksheet "Site">

In [67]:
row_count = ws.max_row
j=1
unique_list = list()
unique_items = 0

SSID = 28356 # assuming all coords in UTM zone 56
qrystr = "INSERT INTO form.field_site(site_label,geom,elevation) values('%s',ST_GeomFromText('POINT(%s %s)', %s),%s) ON CONFLICT DO NOTHING"

# connect to the PostgreSQL server
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(**params)

ws.cell(row=1, column=j).value
for i in range(2, row_count + 1):
    item = ws.cell(row=i, column=j).value
    if item not in unique_list and item is not None:
        unique_list.append(item)
        unique_items += 1
        x=ws.cell(row=i,column=2).value
        y=ws.cell(row=i,column=3).value
        z=ws.cell(row=i,column=5).value
        cur = conn.cursor()
        cur.execute(qrystr % (item,x,y,SSID,z))
        updated_rows = cur.rowcount
        if updated_rows > 0:
            print("%s : %s rows updated" % (item,updated_rows))
        conn.commit()
        cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Connecting to the PostgreSQL database...
Database connection closed.


In [64]:
ws.cell(row=5,column=3).value

'6302797.05803'

In [65]:
unique_items

20

In [16]:
conn = psycopg2.connect(**params)

cur = conn.cursor()
cur.execute("SELECT site_label,location_description,elevation FROM form.field_site WHERE site_label='SCCJB13';")
site_info = cur.fetchone()

cur.close()
conn.close()

site_info


('SCCJB13',
 'Yatteyattah NR, along power easement from northern boundary, and about 40m west of the power stauntion inside the forest edge on the upper slope of the gully',
 None)

In [17]:
site_info[0]


'SCCJB13'

In [22]:
site_info[0].length

AttributeError: 'str' object has no attribute 'length'